In [2]:
import numpy as np
import pandas as pd
import ipywidgets as widgets

from IPython.display import display

# from texttable import Texttable
# 想使用tb=Texttable()输出带表格框线的，但是感觉也没用....

# dot matrix

In [161]:
def dot_matrix(x,y,windows=0):
    # 输入xy形式如'CURREN'
    len1=len(x)
    len2=len(y)
    seq1=list(x)
    seq2=list(y)
    df=pd.DataFrame(np.zeros(len1*len2).reshape(len1,len2),index=seq1,columns=seq2)
    df=df.astype(int)
    if windows==0:
        for i in seq1:
            for j in seq2:
                if i==j:
                    df.loc[i,j]='*'
                    
    else:
        half_windows=int(windows/2)
        for i in range(half_windows,len1-half_windows-windows%2):
            for j in range(half_windows,len2-half_windows-windows%2):
                flag_=0
                m=half_windows
                while m>=0-half_windows-(windows%2-1):
                    if seq1[i-m]==seq2[j-m]:
                        flag_=flag_+1
                        m=m-1
                    else:
                        break
                if flag_==windows:
                    df.loc[seq1[i],seq2[j]]='*'
    
    
    
    df=df.replace(0," ")
    print(df)
    # display(df)
    print("seq1:{},长度:{}".format(x,len1))
    print("seq2:{},长度:{}".format(y,len2))
    
    
print('请输入第1个序列：')
x=input()
print('请输入第2个序列：')
y=input()
print('请输入窗口长度：')
w=int(input())
    
dot_matrix(y,x,w)

请输入第1个序列：
CURRENTLYTROPICAL
请输入第2个序列：
CURRENTTOPICS
请输入窗口长度：
4
   C  U  R  R  E  N  T  L  Y  T  R  O  P  I  C  A  L
C                                                   
U                                                   
R        *  *                    *                  
R        *  *                    *                  
E              *                                    
N                 *                                 
T                                                   
T                                                   
O                                                   
P                                                   
I                                         *         
C                                                   
S                                                   
seq1:CURRENTTOPICS,长度:13
seq2:CURRENTLYTROPICAL,长度:17


# dynamic programming

In [3]:
def dp_global(x,y,match=1,dismatch=0,gap=0,mode='g'):
    # 输入xy形式如'CURREN'
    len1=len(x)
    len2=len(y)
    seq1=list(x)
    seq1.insert(0,"λ")
    seq2=list(y)
    seq2.insert(0,"λ")
    df=pd.DataFrame(np.linspace(99,99,(len1+1)*(len2+1)).reshape(len1+1,len2+1),index=seq1,columns=seq2)
    df=df.astype(int)
    
    df.iat[0,0]=0
    dic_={True:match,False:dismatch}
    for i in range(1,len2+1):
        df.iloc[0].iat[i]=df.iloc[0].iat[i-1]+gap
    for j in range(1,len1+1):
        df.iloc[j].iat[0]=df.iloc[j-1].iat[0]+gap
    for j in range(1,len1+1):
        for i in range(1,len2+1):
            s1=df.iloc[j-1].iat[i-1]+dic_[seq1[j]==seq2[i]]
            s2=df.iloc[j-1].iat[i]+gap
            s3=df.iloc[j].iat[i-1]+gap
            df.iloc[j].iat[i]=max(s1,s2,s3)
    
    j=len1
    i=len2
    res=[[seq1[-1]],[seq2[-1]]]
    
    df=df.astype(str)
    while i>0 and j>0:
        if int(df.iloc[j-1].iat[i-1])+dic_[seq1[j]==seq2[i]] == int(df.iloc[j].iat[i]):
            df.iloc[j].iat[i]="↖"+df.iloc[j].iat[i]
            i=i-1
            j=j-1
            res[0].insert(0,seq1[j])
            res[1].insert(0,seq2[i])
        elif int(df.iloc[j-1].iat[i])+gap == int(df.iloc[j].iat[i]):
            df.iloc[j].iat[i]="↑"+df.iloc[j].iat[i]
            res[0].insert(0,seq1[j-1])
            res[1].insert(1,"-")
            i=i
            j=j-1
        elif int(df.iloc[j].iat[i-1])+gap == int(df.iloc[j].iat[i]):
            df.iloc[j].iat[i]="← "+df.iloc[j].iat[i]
            res[0].insert(1,"-")
            res[1].insert(0,seq2[i-1])
            i=i-1
            j=j
            
    if i!=0:
        if int(df.iloc[j].iat[0])+gap == int(df.iloc[j].iat[i]):
            df.iloc[j].iat[i]="← "+df.iloc[j].iat[i]
    if j!=0:
        if int(df.iloc[0].iat[i])+gap == int(df.iloc[j].iat[i]):
            df.iloc[j].iat[i]="↑"+df.iloc[j].iat[i]

    
    display(df)
    if res[0][0]=="λ" and res[1][0]=="λ":
        res=[res[0][1:],res[1][1:]]
    elif res[0][0]=="λ":
        res[0][0]="-"
    elif res[1][0]=="λ":
        res[1][0]="-"
    
    max_res=max(len(res[0]),len(res[1]))
    df2=pd.DataFrame(np.linspace(99,99,3*max_res).reshape(3,max_res),index=["seq1"," ","seq2"])
    df2.iloc[2]=res[0]
    df2.iloc[0]=res[1]
    for m in range(max_res):
        if df2.iloc[0].at[m]==df2.iloc[2].at[m]:
            df2.iloc[1].at[m]="|"
        else:
            df2.iloc[1].at[m]=" "
    
    display(df2)
    print("".join(res[1]))
    print("".join(res[0]))

In [4]:
def dp_local(x,y,match=1,dismatch=0,gap=0,mode='g'):
    # 输入xy形式如'CURREN'
    len1=len(x)
    len2=len(y)
    seq1=list(x)
    seq1.insert(0,"λ")
    seq2=list(y)
    seq2.insert(0,"λ")
    df=pd.DataFrame(np.linspace(99,99,(len1+1)*(len2+1)).reshape(len1+1,len2+1),index=seq1,columns=seq2)
    df=df.astype(int)
    
    df.iat[0,0]=0
    dic_={True:match,False:dismatch}
    for i in range(1,len2+1):
        df.iloc[0].iat[i]=max(df.iloc[0].iat[i-1]+gap,0)
    for j in range(1,len1+1):
        df.iloc[j].iat[0]=max(df.iloc[j-1].iat[0]+gap,0)
    for j in range(1,len1+1):
        for i in range(1,len2+1):
            s1=df.iloc[j-1].iat[i-1]+dic_[seq1[j]==seq2[i]]
            s2=df.iloc[j-1].iat[i]+gap
            s3=df.iloc[j].iat[i-1]+gap
            df.iloc[j].iat[i]=max(s1,s2,s3,0)
    
    max_score=df.stack().max()
    print('max score:{}'.format(max_score))
    display(df)


In [5]:
print('请输入第1个序列：')
x=input()
print('请输入第2个序列：')
y=input()
print('请输入分数(格式为match,dismatch,gap)：')
s=input().split(",")
print('请输入g为global，l为local')
m=input()
if m=="g":
    dp_global(y,x,int(s[0]),int(s[1]),int(s[2]))
if m=="l":
    dp_local(y,x,int(s[0]),int(s[1]),int(s[2]))

请输入第1个序列：
CTCGCAGC
请输入第2个序列：
CATTCAC
请输入分数(格式为match,dismatch,gap)：
1,-1,-5
请输入g为global，l为local
g


,λ,C,T,C,G,C,A,G,C
λ,0,-5,-10,-15,-20,-25,-30,-35,-40
C,-5,↖1,-4,-9,-14,-19,-24,-29,-34
A,-10,-4,↖0,-5,-10,-15,-18,-23,-28
T,-15,-9,-3,↖-1,-6,-11,-16,-19,-24
T,-20,-14,-8,-4,↖-2,-7,-12,-17,-20
C,-25,-19,-13,-7,-5,↖-1,-6,-11,-16
A,-30,-24,-18,-12,-8,-6,↖0,← -5,-10
C,-35,-29,-23,-17,-13,-7,-5,-1,↖-4


,0,1,2,3,4,5,6,7
seq1,C,T,C,G,C,A,G,C
,|,,,,|,|,,|
seq2,C,A,T,T,C,A,-,C


CTCGCAGC
CATTCA-C
